In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pydicom
import pynetdicom

import pymedphys.labs.serviceplans.create
import pymedphys._dicom.header

In [ ]:
machine_map = {
#     '2619': {
#         'sender_ae_title': b'CMS_SCU',
#         'ip': '192.168.100.200',
#         'port': 104
#     },
#     '2694': {
#         'sender_ae_title': b'CMS_SCU',
#         'ip': '192.168.100.201',
#         'port': 104
#     },
#     '4299': {
#         'sender_ae_title': b'PYMEDPHYS',
#         'ip': '127.0.0.1',
#         'port': 44104
#     },
    '9002': {
        'sender_ae_title': b'PYMEDPHYS',
        'ip': '127.0.0.1',
        'port': 43104
    },
}

In [ ]:
full_dose_rates = {
#     '00_06MV': 600,
    '01_10MV': 600,
#     '02_06FFF': 1560,
#     '03_10FFF': 2300
}

half_dose_rates = {
    key: item // 2 for key, item in full_dose_rates.items()
}
eighth_dose_rates = {
    key: item // 8 for key, item in full_dose_rates.items()
}

dose_rate_options = [full_dose_rates, half_dose_rates, eighth_dose_rates]

In [ ]:
plans = {}

for dose_rate_lookup in dose_rate_options:
    for key, dose_rate in dose_rate_lookup.items():
        prepend, energy = key.split('_')
        plans[(energy, dose_rate)] = pymedphys.labs.serviceplans.create.main(energy, dose_rate, prepend=prepend)

In [ ]:
for machine_id, settings in machine_map.items():
    print(f"\n\n========\n  {machine_id}  \n========\n")
    
    ae = pynetdicom.AE(ae_title=settings['sender_ae_title'])
    ae.requested_contexts = pynetdicom.VerificationPresentationContexts
    ae.add_requested_context(
        pynetdicom.sop_class.RTPlanStorage,
        transfer_syntax=pydicom.uid.ExplicitVRLittleEndian
    )
    
    for key, plan in plans.items():
        print(key)
        
        adjusted_plan = pymedphys._dicom.header.adjust_machine_name(plan, machine_id)

        adjusted_plan.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
        adjusted_plan.fix_meta_info(enforce_standard=True)

        assoc = ae.associate(settings['ip'], settings['port'], ae_title=b'EOS_RTD')
        status = assoc.send_c_store(adjusted_plan)
        assoc.release()
        print(pynetdicom.status.code_to_category(status.Status))